# Chapter 7: n-step Bootstrapping

- n-step TD methods generalize both MC and one-step TD methods so that one can shift from one to the other smoothly as needed to meet the demands of a particular task
- In many applications one wants to be able to update the action very fast to take into account anything that has changed, but boostrapping works best if it is over a length of time in which a significant and recognizable state change has occured.
- n-step mehtods enable bootstrapping to occur over multiple steps, freeing us from the tyranny of hte single time step

# 7.1 n-step TD Prediction

- MC methods perform an update for each state based on the entire sequence of observed rewards from that state until the end of the episode
- The update of one-step TD methods is based on just the one next reward, bootstrapping from the value of the stae one step later as a proxy for the remaining rewards.
- Methods that use n-step updates are still TD methods because they still change an earlier estimate based on how it differs from a later estimate. Now the later estimate is just n steps later.  Methods in which the temporal difference extends over n steps are called n-step TD methods
- Terminology: $G_{t}$ is called the target of the update
    - In MC updates the target is the total return over all time steps
    - In one-step updates the target is the first reward plus the discounted estimated value of the next state, which is called the one-step return $$G_{t:t+1} \doteq R_{t+1} + \gamma V_{t}(S_{t+1})$$
- The target for an arbitrary n-step update is the n-step return: $$ G_{t:t+n} \doteq R_{t+1} + \gamma R_{t+2} + ... + \gamma^{n-1}R_{t+n} + \gamma^{n}V_{t+n-1}(S_{t+n})$$ for all n, t s.t. n >= 1 and 0<= t<= T-n
- All n-step returns can be considered approximations to the full return, truncated after n steps and then corrected for the remaining missing terms by $ V_{t+n-1}(S_{t+n}) $
- The natural state-value learning algorithm for using n-step returns is thus: $$ V_{t+n}(S_{t}) \doteq V_{t+n-1}(S_{t}) + \alpha[G_{t:t+n} - V_{t+n-1}(S_{t})]$$ for 0 <= t <= T
- Note that no changes at all are mae during the first n-1 steps because it has not seen the n states yet.  To make up for that, an equal number of additional updates are made at the end of the epidose, after termination and befrore starting the next episode
- Error reduction property of n-step returns: an important property of n-step returns is that thir expectation is guaranteed to be a better estimate of $v_{\pi}$ than $V_{t-n+!}$ is, in a worst-state sense. Theat is, the worst error of the expected n-step return is guaranteed to be less than or equal to $\gamma^{n}$ time the worst error under $V_{t+n-1}$
    - Because of the error reduction property, one can fomally show that all n-step TD methods converge to the correct predictions under appropriate technical conditions.
- The generalization of TD and MC methods to n-step methods can potentially perform better than either of the two extreme methods
    - n appears to be a hyperparamter that you tune 

In [ ]:
# PSEUSOCODE: n-step TD for estimating V ~= v_PI
"""
Input:  a policy PI
Algorithm paramaters: step size alpha in (0,1], a positive integer n
Initialize V(s) arbitrarily, for all s in S
All store and access operations (for S_t and R_t) can take their index mod n+!

Loop for each episode:
    Initialize and store S_0 /= terminal
    T <- inf
    Loop for t = 0,1,2,...:
        If t < T, then:
            Take an action according to PI(.|S_t)
            Observe and store the reward as R_(t+1) and the next state as S_(t+1)
            If S_(t+1) is terminal, then T <- t+1
        tau <- t-n+1 (tau is the time whose state's estimate is being updated)
        if tau >= 0:
            G <- SUM_(i = tau + 1)->min(tau + n, T) gamma^(i-tau-1)R_i
            If tau + n < T, then G <- G + gamma^n V(S_(tau+n))
            V(S_tau) <- V(S_tau) + alpha[G - V(S_tau)]
    Until tau = T-1

"""

# 7.2 n-step Sarsa

- The n-step version of Sarsa we call n-step Sarsa
- The main idea is to switch states for actions(state-action pairs) and then use an epsilon-greedy policy
- We redefine n-step returns(update targets) in terms of estimated action values: 
$$ G_{t:t+n} \doteq R_{t+1} + \gamma R_{t+2} + ... + \gamma^{n-1}R_{t+n} + \gamma^{n}Q_{t+n-1}(S_{t+n}, A_{t+n})$$ 
n >= 1, 0 <= t < T-n
- The natural algorithm is then: 
$$Q_{t+n}(S_{t}, A_{t}) \doteq Q_{t+n-1} (S_{t}, A_{t}) + \alpha[G_{t:t+n} - Q_{t+n-1}(S_{t}, A_{t})]$$ 0 <= t <= T
- n-step methods can speedup policy learning because n-step methods strengthens the last n action values of the sequence, so that much more is learned from the one episode. 

In [ ]:
# PSEUDOCODE: n-step Sarsa for estimating Q~= q_* or q_PI
"""
Initialize Q(s,a) arbitrarily, for all s in S, a in A
Initialize PI to be eps-greedy w.r.t. Q, or to a fixed given policy
Algorithm parameters: step size alpha in (0,1], small eps > 0, a positive integer n
All store and access operations(for S_t, A_t, R_t) can take their index mod n+1

Loop for each episode:
    Initialize and store S_0 /= terminal
    Select and store an action A_0 ~ PI(.|S_0)
    T <- inf
    Loop for t = 0,1,2,...:
        If t < T, then:
            Take action A_t
            Observe and store the next reward as R_(t+1) and the next state as S_(t+1)
            If S_(t+1) is terminal, then:
                T <- T+1
            else:
                Select and store an action A_{t+1} ~ PI(.|S_(t+1))
        tau <- t - n+1
        If tau >= 0:
            G <- SUM_i=(tau+1)->min(tau+n, T) gamma^(i-tau-1) R_i
            If tau +n < T, then G <- G + gamma^n(S_(tau+n), A_(tau+n))
            Q(S_tau, A_tau) <- Q(S_tau, A_tau) + alpha[G - Q(S_tau, A_tau)]
            If PI is being learned, then ensure that PI(.|S_tau) is eps-greedy w.r.t. Q
        Until tau = T-1

"""

# 7.3 n-step Off-policy Learning

- Recall that off-poliocy learning is learning the value function for one policy, $\pi$, while following another policy, $b$. Often, $\pi$ is the greedy policy for the current action-value function estimate, and $b$ is a more exploratory policy, perhaps eps-greedy.
- In order to use the data from $b$ we must take into account the differences between the policies, using their relative probability of taking the actions that were taken.
    - In n-step methods, returns are constructed over n steps, so we are interested in the relative probability of just those n actions
- The off-policy version of n-step TD has the following update where $\rho_{t:t+n-1}$ is the importancef-sampling ratio:
$$ V_{t+n}(S_{t}) \doteq V_{t+n-1}(S_{t}) + \alpha \rho_{t:t+n-1}[G_{t:t+n} - V_{t+n-1}(S_{t})]$$ 0 <= t<= T
- The importance sampling ratio is the relative probability under the two policyes of taking the n actions from $A_{t}$  to $A_{t+n-1}$:
$$ \rho_{t:h} \doteq \prod\limits_{k=t}^{min(h,T-1)} \frac {\pi(A_{k}|S_{k})} {b(A_{k}|S_{k})}$$

### Rest of chapter is about a range of different n-step TD methods, we might be able to use eligibility traces instead...